In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import *
from sklearn.model_selection import train_test_split

In [0]:
train_data = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/dataset/0831-15-train.csv")
label_data = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/dataset/train_label.csv")
train_p = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/dataset/train_payment.csv")
test1 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/dataset/0903-13-test1.csv")
test2 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/dataset/0903-13-test2.csv")

In [0]:
# train_payment 파일에서 결제 금액이 있는 날은 해당 값으로, 없으면 0으로해서 target data 만들기  (평균으로 넣어서 시도도 해봐야 할 것 같긴한데....)
y_train = train_data.merge(train_p, on=["acc_id", "day"], how="left")["amount_spent"]
y_train.fillna(0, inplace=True)
y_train = np.reshape(y_train.values, (-1, 28, 1))
y_train.shape

(40000, 28, 1)

In [0]:
# From DF to Array
X_train = np.array(train_data.pivot_table(index=["acc_id", "day"]).to_xarray().to_array())
X_train = X_train.transpose(1, 2, 0)
X_train.shape

(40000, 28, 50)

In [0]:
X_test1 = np.array(test1.pivot_table(index=["acc_id", "day"]).to_xarray().to_array())
X_test1 = X_test1.transpose(1, 2, 0)
print(X_test1.shape)

X_test2 = np.array(test2.pivot_table(index=["acc_id", "day"]).to_xarray().to_array())
X_test2 = X_test2.transpose(1, 2, 0)
print(X_test2.shape)

(20000, 28, 50)
(20000, 28, 50)


In [0]:
# # # Split the data for validation
val_size = 0.2
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size)

In [0]:
# Hyperparameters Declaration
n_inputs = X_train.shape[2]
n_steps = X_train.shape[1]
n_outputs = y_train.shape[-1]
n_neurons = 32
n_epochs = 5001
batch_size = X_train.shape[0]   # 
num_accounts = X_train.shape[0]
num_batch = num_accounts // batch_size
learning_rate = 0.05

### Model Structure ###
# Flow: Multi-LSTM -> FC layer(result)
# Cost function: Mean Squared Error
# Optimizer: Adam Gradient Descent

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, shape=[None, n_steps, n_outputs])

# Create RNN Cell
cell = tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

# Variables for 1st fully connected layer
W = tf.Variable(tf.random_normal([n_neurons, n_outputs]))
b = tf.Variable(tf.random_normal([n_outputs]))

# Build the FC layer for hypothesis function
hf = tf.matmul(outputs, W) + b

# Declare the model
loss = tf.reduce_mean(tf.square(hf - y))
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

W0904 12:22:23.037941 140376411367296 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0904 12:22:23.039674 140376411367296 deprecation.py:323] From <ipython-input-11-4914ca2d59bc>:24: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0904 12:22:23.043269 140376411367296 deprecation.py:323] From <ipython-input-11-4914ca2d59bc>:27: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Ple

In [0]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        for i in range(num_batch):
            idx = i * batch_size
            X_batch = X_train[idx:idx+batch_size]
            y_batch = y_train[idx:idx+batch_size]
            _, _loss = sess.run([training_op, loss], feed_dict={X: X_batch, y: y_batch})
        
        if epoch % 100 == 0:
            val_loss = sess.run(loss, feed_dict={X: X_val, y: y_val})
            print(f"Epoch: {epoch} \t| MSE: {_loss:.8f} \t| Validation Loss: {val_loss:.8f}")
            
    test_index = np.random.randint(0, int(num_accounts * val_size), 15)
    y_pred = sess.run(hf, feed_dict={X: X_val[test_index]})
    y_true = y_val[test_index]
    
    for p, t in zip(y_pred, y_true):
        print("=" * 40)
        for _p, _t in zip(p, t):
            print(f"Predicted: {_p[0]:.8f}, \tActual: {_t[0]:.8f}")
            
    sequence_data = [sess.run(hf, feed_dict={X:X_test1})]        

Epoch: 0 	| MSE: 2.95428228 	| Validation Loss: 11.01399231
Epoch: 100 	| MSE: 0.07253240 	| Validation Loss: 0.07240256
Epoch: 200 	| MSE: 0.07172926 	| Validation Loss: 0.07162109
Epoch: 300 	| MSE: 0.07132297 	| Validation Loss: 0.07123533
Epoch: 400 	| MSE: 0.07100970 	| Validation Loss: 0.07094292
Epoch: 500 	| MSE: 0.07070521 	| Validation Loss: 0.07062881
Epoch: 600 	| MSE: 0.07044382 	| Validation Loss: 0.07038170
Epoch: 700 	| MSE: 0.07018390 	| Validation Loss: 0.07013386
Epoch: 800 	| MSE: 0.06986912 	| Validation Loss: 0.06979132
Epoch: 900 	| MSE: 0.06971558 	| Validation Loss: 0.06970534
Epoch: 1000 	| MSE: 0.06951969 	| Validation Loss: 0.06952418
Epoch: 1100 	| MSE: 0.06953939 	| Validation Loss: 0.06944982
Epoch: 1200 	| MSE: 0.06932878 	| Validation Loss: 0.06930622
Epoch: 1300 	| MSE: 0.06919198 	| Validation Loss: 0.06914142
Epoch: 1400 	| MSE: 0.06906467 	| Validation Loss: 0.06908613
Epoch: 1500 	| MSE: 0.06891565 	| Validation Loss: 0.06899198
Epoch: 1600 	| MSE:

In [0]:
# Predict values for the other 64 days 
# Input: 28-day-long data per each account predicted from the previous LSTM model
# How the model works:
#      train on first 27 days -> predict 28th day
#      then day by day, 2~28 -> predict 29
#                       3~29 -> predict 30
#                         ...
#                       37~63 -> predict 64

sequence_data = sequence_data[0]

tf.reset_default_graph()

pred_steps = 27
test_num_accounts = X_test1.shape[0]

pred_X = tf.placeholder(tf.float32, shape=[None, pred_steps, n_outputs])
pred_y = tf.placeholder(tf.float32, shape=[None, n_outputs])

pred_cell = tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons)
pred_outputs, pred_states = tf.nn.dynamic_rnn(pred_cell, pred_X, dtype=tf.float32, )

pred_W = tf.Variable(tf.random_normal([n_neurons, n_outputs]))
pred_b = tf.Variable(tf.random_normal([n_outputs]))

index = tf.range(0, 1) * pred_steps
pred_outputs = tf.gather(tf.reshape(pred_outputs, [-1, n_neurons]), index)

pred_hf = tf.matmul(pred_outputs, pred_W) + pred_b

pred_loss = tf.reduce_mean(tf.square(pred_hf - pred_y))
pred_optimizer = tf.train.AdamOptimizer(learning_rate)
pred_training_op = optimizer.minimize(pred_loss)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    result = []
    for i in range(test_num_accounts):
        tmp_sequence = sequence_data[i]
        for epoch in range(15):
                sess.run(pred_training_op, 
                         feed_dict={pred_X: tmp_sequence[:pred_steps].reshape(1, -1, 1), 
                                    pred_y: tmp_sequence[pred_steps].reshape(-1, 1)}) # Do not give an effort to understand this reshaping part. I don't even know.
        for j in range(64-n_steps):
            pred = sess.run(pred_hf, 
                            feed_dict={pred_X: tmp_sequence[j+1:j+pred_steps+1].reshape(1, -1, 1)})
            tmp_sequence = np.append(tmp_sequence, pred[0])
            
        if i % 200 == 0:
            print(f"Progress: {i/test_num_accounts:.2%}")
            
        result.append(np.mean(tmp_sequence))

Progress: 0.00%
Progress: 1.00%
Progress: 2.00%
Progress: 3.00%
Progress: 4.00%
Progress: 5.00%
Progress: 6.00%
Progress: 7.00%
Progress: 8.00%
Progress: 9.00%
Progress: 10.00%
Progress: 11.00%
Progress: 12.00%
Progress: 13.00%
Progress: 14.00%
Progress: 15.00%
Progress: 16.00%
Progress: 17.00%
Progress: 18.00%
Progress: 19.00%
Progress: 20.00%
Progress: 21.00%
Progress: 22.00%
Progress: 23.00%
Progress: 24.00%
Progress: 25.00%
Progress: 26.00%
Progress: 27.00%
Progress: 28.00%
Progress: 29.00%
Progress: 30.00%
Progress: 31.00%
Progress: 32.00%
Progress: 33.00%
Progress: 34.00%
Progress: 35.00%
Progress: 36.00%
Progress: 37.00%
Progress: 38.00%
Progress: 39.00%
Progress: 40.00%
Progress: 41.00%
Progress: 42.00%
Progress: 43.00%
Progress: 44.00%
Progress: 45.00%
Progress: 46.00%
Progress: 47.00%
Progress: 48.00%
Progress: 49.00%
Progress: 50.00%
Progress: 51.00%
Progress: 52.00%
Progress: 53.00%
Progress: 54.00%
Progress: 55.00%
Progress: 56.00%
Progress: 57.00%
Progress: 58.00%
Progres

In [0]:
print(result)

[-0.07093634, -0.011916071, 0.025731185, 0.025767088, 0.09260977, 0.13538253, 0.004987416, 0.012293844, 0.06740934, 0.0102356905, 0.037052605, 0.022130102, 0.01632965, 0.053038467, 0.024193078, 0.02239395, 0.012384307, 0.06524554, 0.022257451, 0.033235043, 0.007813321, 0.02569795, 0.063715085, 0.021839947, 0.04224655, 0.09515049, 0.058986053, 0.04979989, 0.020587686, 0.029379427, 0.043660328, 0.036368724, 0.02665579, 0.3566319, 0.14317223, 0.010529382, 0.045851924, 0.030021397, -0.0056803417, 0.07746138, 0.009994464, 0.0049717203, 0.008202398, 0.026834596, 0.020145832, 0.015439601, 0.027512543, 0.057454824, 0.07688218, 0.0026289984, 0.074512914, 0.02434389, 0.013676556, 0.027198946, 0.097694084, 0.0011554258, -0.00022886437, 0.20004988, 0.196617, -0.024302816, 0.015522802, 0.06816123, 0.018046897, 0.0673821, -0.028130682, 0.014493954, 0.0953413, 0.02635524, 0.003103816, 0.05575199, 0.06652918, 0.038160853, 0.02041323, 0.014565951, 0.053212248, 0.08805314, 0.023957891, 0.008696314, 0.02